In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import modin.pandas as pd
from fastai.tabular import *
from utils import isVaildDate, invalid_date

2019-05-14 20:36:46,288	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-14_20-36-46_4116/logs.
2019-05-14 20:36:46,392	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:18339 to respond...
2019-05-14 20:36:46,499	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:48894 to respond...
2019-05-14 20:36:46,501	INFO services.py:804 -- Starting Redis shard with 10.0 GB max memory.
2019-05-14 20:36:46,509	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-14_20-36-46_4116/logs.
2019-05-14 20:36:46,511	WARNING services.py:1304 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 33736691712 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-

 ## 数据清洗

In [3]:
root = Path('../test_A')
root
root.ls()

PosixPath('../test_A')

[PosixPath('../test_A/ad_static_feature.out'),
 PosixPath('../test_A/ad_operation.dat'),
 PosixPath('../test_A/.ipynb_checkpoints'),
 PosixPath('../test_A/imps_log'),
 PosixPath('../test_A/test_sample.dat'),
 PosixPath('../test_A/user')]

In [4]:
u_data = root/'user'
u_data.ls()
log_data = root/'imps_log'
log_data.ls()

[PosixPath('../test_A/user/user_data')]

[PosixPath('../test_A/imps_log/totalExposureLog.out')]

### 广告静态数据

- 清理掉空值行 
- **空值的广告从出题意图上来说就没打算让选手作为训练数据，可以清洗掉**
- 存在多值的情况

In [5]:
col_names = ['广告id', '创建时间', '广告账户id', '商品id', '商品类型', '广告行业id', '素材尺寸']

In [6]:
ad_static = pd.read_csv(root/'ad_static_feature.out', sep='\t', header=None, names=col_names, low_memory=False)

In [7]:
ad_static.describe()

,广告id,创建时间,广告账户id,商品类型
count,735911.00000,7.359110e+05,735911.000000,735911.000000
mean,368025.70731,1.531054e+09,14582.130906,9.288659
std,212477.19597,1.731993e+08,8537.067157,6.922933
min,1.00000,0.000000e+00,1.000000,1.000000
25%,184024.50000,1.550719e+09,7438.500000,3.000000
50%,368018.00000,1.551760e+09,14066.000000,8.000000
75%,552034.50000,1.552622e+09,22030.000000,18.000000
max,736053.00000,1.553522e+09,29743.000000,18.000000


In [8]:
ad_static.shape

(735911, 7)

- 删除含有空值的行

商品id: 广告推广目标的唯一标识,若推广目标是落地页,则该字段为空, 空值是有效值

In [9]:
any(ad_static['商品id'].isna())  # 不存在空值

False

In [10]:
ad_static.dropna(axis=0, how='any', inplace=True)

In [11]:
ad_static.shape

(509252, 7)

In [12]:
ad_static.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
1,233649,1538221936,25681,7356,13,136,1
2,547531,1550731020,20696,-1,1,186,40
3,707841,1551857857,3968,-1,3,186,40
5,733436,1552977426,22405,31722,5,117,64
6,249105,1552641796,11360,29999,18,145,44


In [13]:
ad_static.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 509252 entries, 1 to 735910
Data columns (total 7 columns):
广告id      509252 non-null int64
创建时间      509252 non-null int64
广告账户id    509252 non-null int64
商品id      509252 non-null object
商品类型      509252 non-null int64
广告行业id    509252 non-null object
素材尺寸      509252 non-null object
dtypes: int64(4), object(3)
memory usage: 31.1+ MB


- 多值处理
    - 广告账户 id:广告所在账户的唯一标识（满足）
    - 商品 id:广告推广目标的唯一标识（不满足）
    - 商品类型:广告推广目标的类型,枚举型（满足）
    - 广告行业 id:广告所属的行业类别标识（不满足）
    - 素材尺寸:不同广告位对素材的尺寸要求不同,同一个广告可能有**多个不同尺寸的素材,用逗号分隔**（满足）

- 删除重复记录广告
- 同一广告 id 不同的设置显然曝光效果可能是有差异的(建议保留)

In [14]:
# ad_static.drop_duplicates(subset='广告id', keep='first', inplace=True)

In [15]:
# ad_static.shape

- 重建索引

In [16]:
ad_static.reset_index(drop=True, inplace=True)

- 直接删除无效行 （商品id， 广告行业id）

In [17]:
# 判断是否存在多值
def is_more(df, field):
    ret, l = [], len(df)
    for i in range(l):
        if len(str(df.loc[i, field]).split(',')) > 1:
            ret.append(i)
    return ret

In [18]:
rm_idx = [is_more(ad_static, f) for f in ['商品id', '广告行业id']]

In [19]:
rm_idx = set(rm_idx[0] + rm_idx[1])
len(rm_idx)

7099

In [20]:
ad_static.drop(axis=0, index=rm_idx, inplace=True)

In [21]:
ad_static.reset_index(drop=True, inplace=True)

In [22]:
ad_static.tail()
ad_static.shape

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
502148,82588,1552967764,21666,21917,18,198,64
502149,665036,1550740115,6184,11633,18,232,64
502150,491914,1548392470,1809,5278,5,232,64
502151,3990,1551428228,4255,3401,18,218,36
502152,222150,1551340613,4250,14527,18,94,36


(502153, 7)

- 数据类型转换

In [23]:
ad_static.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502153 entries, 0 to 502152
Data columns (total 7 columns):
广告id      502153 non-null int64
创建时间      502153 non-null int64
广告账户id    502153 non-null int64
商品id      502153 non-null object
商品类型      502153 non-null int64
广告行业id    502153 non-null object
素材尺寸      502153 non-null object
dtypes: int64(4), object(3)
memory usage: 26.8+ MB


In [24]:
ad_static[['商品id', '广告行业id']] = ad_static[['商品id', '广告行业id']].apply(pd.to_numeric)

In [25]:
ad_static.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502153 entries, 0 to 502152
Data columns (total 7 columns):
广告id      502153 non-null int64
创建时间      502153 non-null int64
广告账户id    502153 non-null int64
商品id      502153 non-null int64
商品类型      502153 non-null int64
广告行业id    502153 non-null int64
素材尺寸      502153 non-null object
dtypes: int64(6), object(1)
memory usage: 26.8+ MB


In [26]:
x = is_more(ad_static, '素材尺寸')

In [27]:
ad_static.loc[x]

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
470194,696620,1550625824,12111,-1,2,170,"17,57"


- 非法日期

- 创建日期为0

In [28]:
ad_static.groupby(by='创建时间').count().head()

,广告id,广告账户id,商品id,商品类型,广告行业id,素材尺寸
创建时间,,,,,,
0,4487,4487,4487,4487,4487,4487
1428399835,1,1,1,1,1,1
1433409092,1,1,1,1,1,1
1457591734,1,1,1,1,1,1
1457591735,2,2,2,2,2,2


In [29]:
ad_static = ad_static.sort_values('创建时间')
ad_static.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
280090,639546,0,9140,3531,8,175,40
200778,719010,0,8010,6880,18,186,44
200662,192083,0,23915,-1,1,21,54
342101,605702,0,29394,30873,18,93,44
200600,531484,0,15556,-1,1,136,14


In [30]:
ad_static.reset_index(drop=True, inplace=True)

In [31]:
ad_static.tail()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
502148,313930,1553005882,2323,12310,8,32,40
502149,101387,1553006951,2111,149,8,91,30
502150,242880,1553006957,10704,4242,8,91,14
502151,26508,1553006962,2323,12310,8,32,40
502152,621709,1553007151,9272,23191,18,186,40


In [32]:
ad_static = ad_static.iloc[4487:, :]

In [33]:
ad_static.reset_index(drop=True, inplace=True)

In [34]:
ad_static.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
0,593323,1428399835,26889,-1,3,85,64
1,146563,1433409092,23511,-1,3,118,64
2,216969,1457591734,26970,-1,1,33,1
3,357401,1457591735,26970,-1,1,33,1
4,542734,1457591735,26970,-1,1,33,34


In [35]:
ad_static = invalid_date(ad_static, '创建时间')

In [36]:
ad_static['创建时间'] = pd.to_datetime(ad_static['创建时间'], unit='s')  # 转为日期

In [37]:
ad_static.head()
ad_static.tail()
ad_static.shape

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
0,593323,2015-04-07 09:43:55,26889,-1,3,85,64
1,146563,2015-06-04 09:11:32,23511,-1,3,118,64
2,216969,2016-03-10 06:35:34,26970,-1,1,33,1
3,357401,2016-03-10 06:35:35,26970,-1,1,33,1
4,542734,2016-03-10 06:35:35,26970,-1,1,33,34


,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
497661,313930,2019-03-19 14:31:22,2323,12310,8,32,40
497662,101387,2019-03-19 14:49:11,2111,149,8,91,30
497663,242880,2019-03-19 14:49:17,10704,4242,8,91,14
497664,26508,2019-03-19 14:49:22,2323,12310,8,32,40
497665,621709,2019-03-19 14:52:31,9272,23191,18,186,40


(497666, 7)

- 数据保存

In [38]:
ad_static.to_csv('../data/ad_static.csv', index=None, encoding='utf-8')